In [2]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import random

In [3]:
with open('drive/MyDrive/classes.txt') as f:
  lines = f.readlines()
random.shuffle(lines)
classes = [lines[i][0:-1].replace(' ', '_') for i in range(100)]
print((classes))

['car', 'mountain', 'washing_machine', 'harp', 'syringe', 'helmet', 't-shirt', 'peanut', 'helicopter', 'knee', 'suitcase', 'flashlight', 'snake', 'submarine', 'bus', 'anvil', 'crocodile', 'skyscraper', 'carrot', 'popsicle', 'teddy-bear', 'horse', 'compass', 'cake', 'hand', 'couch', 'peas', 'rifle', 'wheel', 'whale', 'matches', 'bread', 'trumpet', 'wristwatch', 'mosquito', 'basketball', 'string_bean', 'crab', 'piano', 'stop_sign', 'broom', 'oven', 'house_plant', 'moustache', 'golf_club', 'van', 'hammer', 'eyeglasses', 'lightning', 'octopus', 'kangaroo', 'book', 'jail', 'garden', 'lipstick', 'hurricane', 'sleeping_bag', 'owl', 'palm_tree', 'elephant', 'lobster', 'cloud', 'fish', 'bulldozer', 'airplane', 'parrot', 'beard', 'basket', 'speedboat', 'strawberry', 'elbow', 'leg', 'lantern', 'hedgehog', 'spider', 'square', 'passport', 'bat', 'house', 'waterslide', 'jacket', 'dresser', 'telephone', 'butterfly', 'mouth', 'computer', 'potato', 'truck', 'umbrella', 'shorts', 'fan', 'hockey_stick', 

In [ ]:
import urllib.request
def download():

  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in classes:
    cls_url = c.replace('_', '%20')
    path = base+cls_url+'.npy'
    print(path)
    urllib.request.urlretrieve(path, c+'.npy')


In [ ]:
download()

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/violin.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/pig.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bowtie.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/animal%20migration.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/hot%20air%20balloon.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/giraffe.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/envelope.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/onion.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/waterslide.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/wine%20glass.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/car.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/canoe.npy
https://storage.googleapis.com/qui

In [ ]:
def load_data(root, vfold_ratio=0.2, max_items_per_class= 5000 ):
    all_files = glob.glob(os.path.join(root, '*.npy'))

    #initialize variables
    x = np.empty([0, 784])
    y = np.empty([0])
    class_names = []

    #load a subset of the data to memory
    for idx, file in enumerate(all_files):
        data = np.load(file)
        data = data[0: max_items_per_class, :]
        labels = np.full(data.shape[0], idx)

        x = np.concatenate((x, data), axis=0)
        y = np.append(y, labels)

        class_name, ext = os.path.splitext(os.path.basename(file))
        class_names.append(class_name)

    data = None
    labels = None

    #separate into training and testing
    permutation = np.random.permutation(y.shape[0])
    x = x[permutation, :]
    y = y[permutation]

    vfold_size = int(x.shape[0]/100*(vfold_ratio*100))

    x_test = x[0:vfold_size, :]
    y_test = y[0:vfold_size]

    x_train = x[vfold_size:x.shape[0], :]
    y_train = y[vfold_size:y.shape[0]]
    return x_train, y_train, x_test, y_test, class_names

In [ ]:
x_train, y_train, x_test, y_test, class_names = load_data('/content')

In [ ]:
image_size = 28
num_classes = 100

In [ ]:
# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Convert class vectors to class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Define model
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(100, activation='softmax'))
# Train model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 32)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 3, 3, 64)          0

In [ ]:
model.fit(x = x_train, y = y_train, validation_split=0.1, batch_size = 256, verbose=2, epochs=5)

#evaluate on unseen data
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuarcy: {:0.2f}%'.format(score[1] * 100))


Epoch 1/5
1407/1407 - 20s - loss: 1.8786 - top_k_categorical_accuracy: 0.7887 - val_loss: 1.3836 - val_top_k_categorical_accuracy: 0.8754 - 20s/epoch - 14ms/step
Epoch 2/5
1407/1407 - 8s - loss: 1.2546 - top_k_categorical_accuracy: 0.8895 - val_loss: 1.2006 - val_top_k_categorical_accuracy: 0.8961 - 8s/epoch - 6ms/step
Epoch 3/5
1407/1407 - 8s - loss: 1.1149 - top_k_categorical_accuracy: 0.9047 - val_loss: 1.1062 - val_top_k_categorical_accuracy: 0.9064 - 8s/epoch - 6ms/step
Epoch 4/5
1407/1407 - 8s - loss: 1.0353 - top_k_categorical_accuracy: 0.9138 - val_loss: 1.0426 - val_top_k_categorical_accuracy: 0.9136 - 8s/epoch - 5ms/step
Epoch 5/5
1407/1407 - 8s - loss: 0.9798 - top_k_categorical_accuracy: 0.9196 - val_loss: 1.0143 - val_top_k_categorical_accuracy: 0.9144 - 8s/epoch - 6ms/step
Test accuarcy: 91.62%


In [ ]:
model.save('line-drawing.h5')

In [ ]:
%pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 24.7 MB/s eta 0:00:00


In [ ]:
%pip install tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=e8b62e10b232b99caf84c34344f4b85a69ce77ae109f693e966a98b09ccd9dde
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


In [ ]:
%pip install tensorflow

In [ ]:
!mkdir model
!tensorflowjs_converter --input_format keras line-drawing.h5 model/

2023-09-14 22:16:02.423857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 22:16:05.932196: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/model.json (deflated 82%)
  adding: model/group1-shard1of1.bin (deflated 7%)


In [ ]:
from google.colab import files
files.download('model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>